In [2]:
import numpy as np
import copy
from io import StringIO
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

In [50]:
dataset=StringIO("""Date,Open,High,Low,Close,Volume,Trade_count,Vwap
2015-12-01 09:00:00+00:00,118.88,118.94,118.88,118.94,1145,5,118.902052
2015-12-01 09:15:00+00:00,118.77,118.77,118.77,118.77,200,1,118.77
2015-12-01 09:30:00+00:00,118.69,118.69,118.6,118.6,900,4,118.61
2015-12-01 09:45:00+00:00,118.64,118.65,118.64,118.65,3580,5,118.648883
2015-12-01 10:00:00+00:00,118.65,118.65,118.55,118.55,1820,4,118.611538
2015-12-01 10:15:00+00:00,118.55,118.6,118.55,118.6,880,5,118.5625
2015-12-01 10:30:00+00:00,118.55,118.55,118.5,118.5,1878,5,118.513312
2015-12-01 10:45:00+00:00,118.59,118.72,118.59,118.72,2499,10,118.628431
2015-12-01 11:00:00+00:00,118.71,118.9,118.71,118.9,2842,11,118.86064
2015-12-01 11:15:00+00:00,118.87,118.87,118.87,118.87,300,2,118.87
2015-12-01 11:30:00+00:00,118.78,118.8,118.76,118.8,3914,22,118.785876
2015-12-01 11:45:00+00:00,118.8,118.99,118.77,118.9,7900,37,118.893542
2015-12-01 12:00:00+00:00,118.88,118.98,118.84,118.84,6540,34,118.922648
2015-12-01 12:15:00+00:00,118.82,118.84,118.77,118.77,5603,28,118.804962
2015-12-01 12:30:00+00:00,118.77,118.89,118.76,118.88,7612,31,118.824002
""")
df = pd.read_table(dataset, sep=",")

#ip = np.array([ [1,2,3],[6,8,9],[3,4,5],[4,7,8],[4,2,5],[5,7,4] ])
#op = np.array([[2,8,4,7,2,4]])
#op = op.reshape(6,1)
ip = np.array([ [1],  [2],  [0],  [2],  [0],  [1],  [2],  [1] ])
op = np.array([ [300],[100],[200],[100],[200],[300],[100],[300] ])
num_steps = 3
num_features = 3
#ip_shaped = np.reshape(ip, newshape=(-1, num_steps, num_features))

#X = np.array([ [1,2,3, 4, 5, 6] ])
#Y = np.array([[2,3,4,5,6,7]])
print(ip)
print("--")
#ip= np.tile(ip,(50,1))
#op = np.tile(op,(50,1))
print(len(op))
ip = np.array([ [1],[1],[1],[1],[1],[2],[1],[2]])
op = np.array([ [1,    0,    1,    0,    1,    1,    0,    1 ]]).T

#ip = np.array([ [1,1,2],[1,2,1],[1,0,2],[1,2,1],[1,0,2],[1,1,2],[1,2,1],[1,1,2]])
#op = np.array([ [1,    0,    1,    0,    1,    1,    0,    1 ]]).T
ip = np.array([ [1,1,1,1,1],[1,1,1,1,0],[1,1,1,1,1],[1,1,1,1,0],[1,1,1,1,1],[1,1,1,1,1],[1,1,1,1,0],[1,1,1,1,1]])
op = np.array([ [1,        0,        1,        0,        1,        1,        0,        1 ]]).T
_, numFeats = ip.shape
num_steps = 3

[[1]
 [2]
 [0]
 [2]
 [0]
 [1]
 [2]
 [1]]
--
8


In [51]:
def lstm_data_transform(x_data, y_data, num_steps=2):
    """ Changes data to the format for LSTM training 
for sliding window approach """
    # Prepare the list for the transformed data
    X, y = list(), list()
    # Loop of the entire data set
    #print(x_data.shape[0])
    for i in range(x_data.shape[0]):
        # compute a new (sliding window) index
        end_ix = i + num_steps

        # if index is larger than the size of the dataset, we stop
        #print(end_ix)
        if end_ix >= x_data.shape[0]:
            break
        # Get a sequence of data for x
        seq_X = x_data[i:end_ix]
        #print(x_data[i:end_ix])
        # Get only the last element of the sequency for y
        #print(y_data[end_ix])
        seq_y = y_data[end_ix-1]
        # Append the list with sequencies
        X.append(seq_X)
        y.append(seq_y)
        #print(X,y)
    # Make final arrays
    x_array = np.array(X)
    y_array = np.array(y)
    return x_array, y_array

In [52]:
ipt,opt=lstm_data_transform(ip,op, num_steps)
print(ipt)
print(ipt.shape)
print(ipt[0].shape)

print("====")
print(opt)

[[[1 1 1 1 1]
  [1 1 1 1 0]
  [1 1 1 1 1]]

 [[1 1 1 1 0]
  [1 1 1 1 1]
  [1 1 1 1 0]]

 [[1 1 1 1 1]
  [1 1 1 1 0]
  [1 1 1 1 1]]

 [[1 1 1 1 0]
  [1 1 1 1 1]
  [1 1 1 1 1]]

 [[1 1 1 1 1]
  [1 1 1 1 1]
  [1 1 1 1 0]]]
(5, 3, 5)
(3, 5)
====
[[1]
 [0]
 [1]
 [1]
 [0]]


In [40]:
# convert output of sigmoid function to its derivative
def sigmoid_output_to_derivative(output):
    return output*(1-output)

# compute sigmoid nonlinearity
def sigmoid(x):
    output = 1/(1+np.exp(-x))
    return output




In [53]:
import numpy as np
hd=200
ipd = num_steps
od = 1

def nonlin(x,deriv=False):
    if(deriv==True):
        return x*(1-x)

    return 1/(1+np.exp(-x))
    
alpha = .2

np.random.seed(3)
print("\nTraining With Alpha:" + str(alpha))

# randomly initialize our weights with mean 0
syn0 = 2*np.random.random((ipd,hd)) - 1
syn1 = 2*np.random.random((hd,od)) - 1
synh = 2*np.random.random((hd,hd)) - 1

syn0upd = np.zeros_like(syn0)
syn1upd = np.zeros_like(syn1)
synhupd = np.zeros_like(synh)


for j in range(10000):
    count = 0
    l2_deltas = list()
    l1_values = list()
    l1_values.append(np.zeros(hd))
    
    d = np.zeros_like(opt)
    overallError = 0


    for inp in ipt:
        X = inp.T
        y = opt[count]
        count+=1
                
        # hidden layer (input ~+ prev_hidden)
        l1 = sigmoid(np.dot(X,syn0) + np.dot(l1_values[-1],synh))
        # output layer (new binary representation)
        l2 = sigmoid(np.dot(l1,syn1))
        
        # how much did we miss the target value?
        l2_error = y - l2
        
        l2_deltas.append((l2_error)*sigmoid_output_to_derivative(l2))
        overallError += np.abs(l2_error[0])
    
        # decode estimate so we can print it out
        d[len(ipt) - count - 1] = np.round(l2[0][0])

        # store hidden layer so we can use it in the next timestep
        l1_values.append(copy.deepcopy(l1))
        
        
    future_l1_delta = np.zeros(hd)
    
    count = 0
    for inp in ipt:
        X= inp.T
        y = opt[count]

        l1 = l1_values[-count-1]
        prev_l1 = l1_values[-count-2]
        
        # error at output layer
        l2_delta = l2_deltas[-count-1]
        # error at hidden layer
        l1_delta = (future_l1_delta.dot(synh.T) + l2_delta.dot(syn1.T)) * sigmoid_output_to_derivative(l1)

        # let's update all our weights so we can try again
        syn1upd += np.atleast_2d(l1).T.dot(l2_delta)
        if l1_delta.shape != prev_l1.shape:
            prev_l1 = np.tile(prev_l1, (numFeats, 1))

        synhupd += np.atleast_2d(prev_l1).T.dot(l1_delta)
        syn0upd += X.T.dot(l1_delta)
        
        future_l1_delta = l1_delta
        count+=1

    syn0 += syn0upd * alpha
    syn1 += syn1upd * alpha
    synh += synhupd * alpha    

    syn0upd *= 0
    syn1upd *= 0
    synhupd *= 0
    
    # print out progress
    if(j % 1000 == 0):
        print(j)
        print("Error:" + str(overallError))
        print("Pred:" + str(d.T))
        print("True:" + str(opt.T))
#         out = 0
#         for index,x in enumerate(reversed(d)):
#             out += x*pow(2,index)
#         print str(a_int) + " + " + str(b_int) + " = " + str(out)
        print("------------")



Training With Alpha:0.2
0
Error:[2.98554331]
Pred:[[0 0 0 0 0]]
True:[[1 0 1 1 0]]
------------
1000
Error:[2.00003855]
Pred:[[1 1 1 1 1]]
True:[[1 0 1 1 0]]
------------
2000
Error:[0.00413604]
Pred:[[1 1 0 1 0]]
True:[[1 0 1 1 0]]
------------
3000
Error:[0.00245247]
Pred:[[1 1 0 1 0]]
True:[[1 0 1 1 0]]
------------
4000
Error:[0.00199782]
Pred:[[1 1 0 1 0]]
True:[[1 0 1 1 0]]
------------
5000
Error:[0.00176695]
Pred:[[1 1 0 1 0]]
True:[[1 0 1 1 0]]
------------
6000
Error:[0.00161981]
Pred:[[1 1 0 1 0]]
True:[[1 0 1 1 0]]
------------
7000
Error:[0.00151379]
Pred:[[1 1 0 1 0]]
True:[[1 0 1 1 0]]
------------
8000
Error:[0.00143148]
Pred:[[1 1 0 1 0]]
True:[[1 0 1 1 0]]
------------
9000
Error:[0.00136441]
Pred:[[1 1 0 1 0]]
True:[[1 0 1 1 0]]
------------
